In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 270 rows, 270 columns and 270 nonzeros
Model has 36585 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+00, 1e+06]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 270 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 270 columns, 0 nonzeros
Presolved model has 36585 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 265
 AA' NZ     : 3.479e+04
 Factor NZ  : 3.507e+04 (roughly 1 MByte of memory)
 Factor Ops : 6.176e+06 (less than 1 second per iteration)
 Threads    : 12

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -9.21241785e+08 -2.21959270e+05  1.79e+04 3.46e+02  1.00e+06     0s
   1  -1.28047956e+08 -7.42509189e+06  2.59e+03 8.71e+01  1.91e+05     0s
   2  -1.00732122e+07 -3.73981752e+06  2.38e+02 1.32e+0

In [2]:
# x = GLS_Apr_weekday_PM.x; # sample matrix, each column is a link flow vector sample; number_of_links * K
# S = GLS_Apr_weekday_PM.S; # sample covariance matrix
# A = GLS_Apr_weekday_PM.A; # link_route incidence matrix
P = GLS_Apr_weekday_PM.P; # route_choice_probability_matrix
# Q = GLS_Apr_weekday_PM.Q; 
L = GLS_Apr_weekday_PM.L; # dimension of xi
number_of_routes = GLS_Apr_weekday_PM.number_of_routes;
number_of_links = GLS_Apr_weekday_PM.number_of_links;

In [3]:
P

90×270 Array{Float64,2}:
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [4]:
# A = sparse(A);
# P = sparse(P);
# Q = sparse(Q);

In [5]:
# K = size(x, 2)

# inv_S = inv(S)

# A_t = transpose(A)

# b = sum([A_t * inv_S * x[:, k] for k = 1:K])

In [6]:
using JuMP

### implement GLS method to estimate OD demand matrix

In [7]:
# model = Model(solver=IpoptSolver())

# @variable(model, xi[1:L] >= 0)

# # Set objective: (K/2) xi' * Q * xi - b' * xi
# obj = 0
# for i = 1:L
#     for j = 1:L
#         obj += (1.0 / 2) * K * xi[i] * Q[i, j] * xi[j]
#     end
# end
# for l = 1:L
#     obj += - b[l] * xi[l]
# end

# @NLobjective(model, Min, obj) 

# solve(model)

In [8]:
# xi_list = getvalue(xi)

xi_list = GLS_Apr_weekday_PM.xi_list

270-element Array{Any,1}:
   2.60532e-11
   4.50565e-11
   5.12524e-11
   3.63218e-11
   1.733e-10  
 311.123      
   1.74958e-11
   7.5618e-11 
 256.938      
 297.969      
   2.9384e-11 
  30.2189     
 575.0        
   ⋮          
   2.84385e-10
   5.37653e-10
   1.01756e-11
   5.25835e-10
   4.45661e-10
   1.23334e-10
   3.09306e-10
   1.28123e-11
   1.47792e-11
   5.4355e-10 
   9.02547e-11
   0.000169428

In [9]:
for idx = 1:L
    if xi_list[idx] < 1e-1
        xi_list[idx] = 0
    end
end

In [10]:
# for idx = 1:L
#     println(xi_list[idx])
# end

0
0
0
0
0
311.1226334883628
0
0
256.937776906513
297.9692146693422
0
30.2189436035768
575.0002872835869
77.2196437154765
0
0
0
204.74267359771923
0
0
0
136.8948059301606
0
0
0
0
0
0
0
72.45155557883656
0
0
424.87526901917147
0
0
0
0
0
133.10373241642583
0
351.2393138256833
0
0
0
0
0
0
0
387.94954224595585
0
21.037036370854352
0
0
0
0
0
80.58300278235774
0
0
239.52023404367176
0
0
0
0
0
0
222.58041131588817
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
276.36869650705177
692.8954683541996
79.42146244663951
11.789870528101318
19.283447181863963
0
0
200.28755855612206
0
0
44.865311591058195
0
0
0
0
0
0
0
0
0
224.90040713620823
28.25563461773596
87.03499266619572
104.85054547595932
0
0
0
157.0475688796781
220.56594254759813
132.79315819327067
0
152.07440416357485
0
768.3134036666553
139.68781265857118
97.79744919015491
0
0
0
0
106.67618298721482
0
0
0
0
745.5740161502831
345.2091744849558
185.0243825577747
0
82.77240937026475
0
0
0
573.2454444783893
0
0
0
0
0
0
0
0
706.7926139686363
0
33.0

In [ ]:
mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)

In [ ]:
getvalue(lam)

In [ ]:
outfile = open("../temp_files/od_demand_vector_simplified_journal_Apr_PM.json", "w")

JSON.print(outfile, getvalue(lam))

close(outfile)

In [ ]:
getobjectivevalue(mGLSJulia)